In [32]:
import pandas as pd
import numpy as np
import re 
trump = pd.read_csv('realdonaldtrump.csv')
temp = []
for index, i in enumerate(trump['date']):
    if i[0:4] in ['2017','2018','2019','2020']:    
        temp.append(index)
pres_trump = trump.loc[temp]

In [4]:
obama = pd.read_csv('Obamawhitehouse_tweets.csv')
biden = pd.read_csv('POTUS_tweets.csv')

In [5]:
print(obama.shape)
print(biden.shape)
print(pres_trump.shape)
# print(obama.head())
# print(biden.head())
# print(pres_trump.head())

(1919, 5)
(2402, 5)
(12235, 8)


In [6]:
# creating the csvs
o_tweets = obama[['text','created_at']]
b_tweets = biden[['text','created_at']]
t_tweets = pres_trump[['content','date']]
t_tweets.rename({'content': 'text', 'date': 'created_at'}, axis=1, inplace=True)
# o_tweets.to_csv('obama.csv',index=False)
# b_tweets.to_csv('biden.csv',index=False)
# t_tweets.to_csv('trump.csv',index=False)

C:\Users\slcro\anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [34]:
#Removing RT, Punctuation etc
remove_rt = lambda x: re.sub('RT @\w+: '," ",x)
rt = lambda x: re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x)
o_tweets["text"] = o_tweets.text.map(remove_rt).map(rt)
o_tweets["text"] = o_tweets.text.str.lower()
b_tweets["text"] = b_tweets.text.map(remove_rt).map(rt)
b_tweets["text"] = b_tweets.text.str.lower()
t_tweets["text"] = t_tweets.text.map(remove_rt).map(rt)
t_tweets["text"] = t_tweets.text.str.lower()

<ipython-input-34-eadb1a2550f7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  o_tweets["text"] = o_tweets.text.map(remove_rt).map(rt)
<ipython-input-34-eadb1a2550f7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  o_tweets["text"] = o_tweets.text.str.lower()
<ipython-input-34-eadb1a2550f7>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

In [35]:
# creating labels using nltk vader
import nltk
# nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer as si
def getSents(textList):
    sents = []
    asi = si()
    for text in textList['text']:
        temp = asi.polarity_scores(text)['compound']
        if temp >= 0.05:
            sents.append('Positive')
        elif (temp > -0.05) and (temp < 0.05):
            sents.append('Neutral')
        elif temp <= -0.05:
            sents.append('Negative')
    return sents


In [36]:
o_tweets['sents'] = getSents(o_tweets)
b_tweets['sents'] = getSents(b_tweets)
t_tweets['sents'] = getSents(t_tweets)


<ipython-input-36-a458eae1d0fa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  o_tweets['sents'] = getSents(o_tweets)
<ipython-input-36-a458eae1d0fa>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b_tweets['sents'] = getSents(b_tweets)


In [37]:
o_tweets.to_csv('obama.csv',index=False)
b_tweets.to_csv('biden.csv',index=False)
t_tweets.to_csv('trump.csv',index=False)

In [38]:
print('Obama:')
print(o_tweets['sents'].value_counts(normalize=True))
print('Trump:')
print(t_tweets['sents'].value_counts(normalize=True))
print('Biden:')
print(b_tweets['sents'].value_counts(normalize=True))

Obama:
Positive    0.544554
Neutral     0.293903
Negative    0.161542
Name: sents, dtype: float64
Trump:
Positive    0.559134
Negative    0.299714
Neutral     0.141152
Name: sents, dtype: float64
Biden:
Positive    0.684846
Negative    0.172773
Neutral     0.142381
Name: sents, dtype: float64


In [41]:
# filter out dataframe for economic terms 
def econ_filter(tweets):
    return tweets.loc[tweets['text'].str.contains('economy|money|stocks|market|bank|economies|jobs|job')]

In [43]:
econ_o_tweets = econ_filter(o_tweets)
econ_t_tweets = econ_filter(t_tweets)
econ_b_tweets = econ_filter(b_tweets)

In [44]:
print('Obama:')
print(econ_o_tweets['sents'].value_counts(normalize=True))
print('Trump:')
print(econ_t_tweets['sents'].value_counts(normalize=True))
print('Biden:')
print(econ_b_tweets['sents'].value_counts(normalize=True))

Obama:
Positive    0.595960
Neutral     0.272727
Negative    0.131313
Name: sents, dtype: float64
Trump:
Positive    0.679148
Negative    0.246696
Neutral     0.074156
Name: sents, dtype: float64
Biden:
Positive    0.714628
Negative    0.165468
Neutral     0.119904
Name: sents, dtype: float64
